In [1]:
from modules.pipeline import Pipeline
import openai
openai.api_key = 'sk-_ynWP8bFt1E-Lg6ZxIAt8YV1fdB8qU1SMkWCqMVRtmT3BlbkFJ_FWv5x5Oz1lT6vhGwUtODMSKo36FCBHBW30fYvD3sA' 

# Set the path to your SQLite database and the PDF documentation
db_file = 'chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
pdf_path = 'Chinook_Database_Enriched_Documentation.pdf'
user_query = "List customers who have purchased tracks from more than 2 different genres."

# Initialize the pipeline
pipeline = Pipeline(db_file, user_query, pdf_path)

# Run the pipeline
analysis = pipeline.run()

print(analysis)


c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Retrieved relevant documents based on similarity threshold of 0.8:
Extracting schema and foreign key relationships...
Extracting keywords using RoBERTa...
Extracted Keywords:
0                      than
1               tracks from
2                      List
3                    genres
4                      have
5            from more than
6                 purchased
7            have purchased
8                 more than
9                      from
10    purchased tracks from
11                 who have
12                from more
13                customers
14                different
15                     more
16         purchased tracks
17           List customers
18       customers who have
19         different genres
20       List customers who
21            customers who
22    have purchased tracks
23         tracks from more
24                   tracks
25                      who
26       who have purchased
dtype: object
Mapping keywords to schema and creating subgraph with v

,CustomerId,FirstName,LastName
0,1,Luís,Gonçalves
1,2,Leonie,Köhler
2,3,François,Tremblay
3,4,Bjørn,Hansen
4,5,František,Wichterlová
5,6,Helena,Holý
6,7,Astrid,Gruber
7,8,Daan,Peeters
8,9,Kara,Nielsen
9,10,Eduardo,Martins


Analyzing result with LLM...

Analysis:
The provided table does not contain information about the genres of tracks purchased by customers. Therefore, it's impossible to list customers who have purchased tracks from more than 2 different genres based on this table.
The provided table does not contain information about the genres of tracks purchased by customers. Therefore, it's impossible to list customers who have purchased tracks from more than 2 different genres based on this table.


In [ ]:
# Run the full pipeline
db_file = 'chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
user_query = " List all employees who have sold tracks to customers in the same city."
run_full_pipeline(user_query, db_file, pdf_file_path)

"""

SELECT Customer.FirstName, Customer.LastName, COUNT(DISTINCT Genre.Name) AS GenreCount
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Genre ON Track.GenreId = Genre.GenreId
GROUP BY Customer.CustomerId
HAVING GenreCount > 2;


"""



In [ ]:
# Run the full pipeline
user_query = " List all employees who have sold tracks to customers in the same city."
run_pipeline_with_semantic_search(user_query, db_file)

"""

SELECT Employee.FirstName, Employee.LastName, Customer.FirstName AS CustomerFirstName, Customer.LastName AS CustomerLastName, Customer.City
FROM Employee
JOIN Customer ON Employee.City = Customer.City
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
WHERE Employee.EmployeeId = InvoiceLine.InvoiceId;



"""

In [ ]:
# Import the necessary libraries
from eralchemy import render_er
from IPython.display import Image
import os

# Get the absolute path to your SQLite database
db_path = os.path.abspath('chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite')

# Form the correct SQLAlchemy URL
db_url = f'sqlite:///{db_path}'

# Generate ERD and save it as a PNG image
output_file = 'chinook_erd.png'
render_er(db_url, output_file)

# Display the generated ERD
img = Image(output_file)
img
